# vector swap



In [157]:
from generate import *
from closest_vs_minimal import *

# def gram_matrix(matrix) -> matrix:
#     return matrix * matrix.transpose()

# def vector_to_list(vector) -> list:
#     return [float(num) for num in vector.list()]

def find_real_minimum_genfile(G, current_row, inserted_component) -> vector:
    """
    returns linear combination of the minimum over real numbers
    the >current_row< parameter is the coordinate of fixed col/row idk
    the >K< parameter is the number inserted into this column
    """
    G = matrix_to_list(G)
    dimension=len(G)
    # print(dimension, "dimension in FRM")
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    # matrixA[0,0] = 1
    # print("G",matrix(G))
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row][j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row][j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    # print("A", matrixA,"\n","b", -(1)*matrixB, )
    result = vector_to_list(matrixA.solve_right((-1)*(inserted_component) * matrixB))
    result.insert(current_row-1, inserted_component)
    return vector(result)

In [158]:
def swapping(basis_as_matrix):
    # basis_as_matrix= matrix([[5, 3, 4], [-2, 4, 4], [-10, -3, -7]]
    # basis_as_matrix = matrix([[3, 10, 4], [-4, 2, 7], [-4, -8, -3]]) # asi funguje??
    # basis_as_matrix = matrix([[-8, -9, 7], [10, 9, -6], [-8, 4, 10]]) # asi funguje??
    # print(basis_as_matrix)
    basis_as_list= matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    print("B = ",basis_as_list)
    used_vectors =[]
    used_vectors.extend(basis_as_list)

    G = gram_matrix(basis_as_matrix)

    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    print("Basis na začatku: ")
    for i in range(dim):
            print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
    print("--------------")
    basis_as_matrix=matrix(basis_as_list)
    # print(basis_as_matrix)
    # print(basis_as_list)

    i = 0
    while i < dim:
#     for j in range(0,dimension):
        # print("\n")
        # print(i)
        real_min = find_real_minimum_genfile(G, i, 1)

        # print(evaluate_norms_at_cube(cube_points(real_min), basis_as_list))
        
        shortestLC = shortest_lc_in_cube(real_min, basis_as_list)
        # print(real_min)
        # print(shortestLC)
        shortest_vector = vector(shortestLC)*basis_as_matrix
        print("sv", shortest_vector,(shortest_vector).norm().n())
        # print("used vectors",used_vectors)
        if vector_to_list(shortest_vector) in used_vectors:
              print("ERROR: I already used the vector ", shortest_vector)
              i+=1
              continue
        if shortest_vector.norm() > basis_as_matrix[-1].norm():
              print("ERROR, LONGER VECTOR")
              i+=1
              continue
        # print(type(shortest_vector))
        shortest_vector = vector_to_list(shortest_vector)
        if shortest_vector == [0 * dim]:
              print("ERROR: Zero vector")
              i+=1
              continue
        used_vectors.append(shortest_vector)
        # print("basis to be thrown away", basis_as_list[dimension-1])
        basis_as_list[dim-1] = shortest_vector
        # print(basis_as_list)
        # print("basis new, should be SV", basis_as_list[dimension-1])
        basis_as_list.sort(key=lambda x: euclidean_norm(x))
        # print(basis_as_list)
        for i in range(dim):
                print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
        basis_as_matrix = matrix(basis_as_list)
        G = gram_matrix(basis_as_matrix)
    print("reduced lll",  (matrix_to_list(matrix(ZZ, basis_as_list).LLL())), euclidean_norm(first_nonzero_norm_vector(matrix_to_list(matrix(ZZ, basis_as_list).LLL()))))
    return euclidean_norm(basis_as_list[0])



 

# swapping(matrix([[-8, -9, 7], [10, 9, -6], [-8, 4, 10]]))

# def swapping_alg_ver2(dimension, perimeter):
#     """ Verse s for cyklem """
#     basis = randomMatrix(dimension, perimeter)
#     i=0
#     while ještě pořád dělám rozumne změny:
#         perform a cube with fixed index i
#         realmin = vysledek cube tedy něco jkao [1, 0.54, 7.2345]
#         sv = zaokrouhleni realmin * B
#         basis[i] = sv
#         i= i + 1 mod dimension
#         basis.sort by length



# CO KDYŽ VE VĚTŠÍCH DIMENZÍCH SE STREFUJU DO VEKTORU KRATŠÍHO NEŽ LLL???? JE TO POSSIBLE !!!! ZKONTROLOVAT
    
# možná se ještě vyplatí opravit cube takto: co když sii jako real-min uložím něco, co ale je moc daleko a  
# fixací JINÉ souřadnice ale dostanu už správnej výsledek? ještě na to mrkni viko

In [159]:
def gaussian_reduction(b_1, b_2):
    #  = basis_as_matrix[0]
    # b_2 = basis_as_matrix[1]
    b_1 = vector(b_1)
    b_2 = vector(b_2)
    if b_1.norm() < b_2.norm():
        b_1 , b_2 = b_2, b_1
    r = round(b_1.dot_product(b_2) / b_2.norm()**2)
    a = b_1 - r * b_2
    if a.norm() < b_2.norm():
        a, b_2 = gaussian_reduction(a, b_2)
    return a, b_2

# gaussian_reduction(Matrix([[1,2],[5,7]]))
# gaussian_reduction(Matrix([[11,4],[19,8]]))
gaussian_reduction([11,4,4],[19,8,1])

((8, 4, -3), (-3, 0, -7))

In [211]:
from sage.modules.free_module_integer import IntegerLattice
from generate import *

def vector_to_int_list(vector) -> list:
    return [int(num) for num in vector.list()]

def semaev(basis_as_matrix):
    # basis_as_matrix = matrix( [[-10, 7, 4], [-7, 2, -5], [8, -9, -6]])

    basis_as_list= matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    # print(dim, "dim")
    # print("B = ",basis_as_list)

    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    # print("Basis na začatku: ")
    # for i in range(dim):
    #         print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
    # print("--------------")


    # Reduce the pair b_1 and b_2
    basis_as_list[1], basis_as_list[0] = gaussian_reduction(basis_as_list[1], basis_as_list[0])
    basis_as_list[1] = vector_to_int_list(basis_as_list[1])
    basis_as_list[0] = vector_to_int_list(basis_as_list[0])
    # print("po gaussově redukci:")
    # for i in range(dim):
    #         print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
    # print("--------------")

    basis_as_matrix=matrix(ZZ,basis_as_list)

    # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2

    G = gram_matrix(basis_as_matrix)
    lc_cube = vector_to_list(find_real_minimum_mistni(G, dim-1, 1))
    # print(lc_cube)
    shortestcube_lc = shortest_lc_in_cube(lc_cube, basis_as_matrix)
    # print(shortestcube_lc)
    a = vector(ZZ,shortestcube_lc) * basis_as_matrix
    print(a, "a před while")

    # if a is shorter than b_3, replace b_3 <- a
    while a.norm().n() < basis_as_matrix[2].norm().n() and vector_to_int_list(a) != [0]*dim and vector_to_int_list(a) not in basis_as_list:
        print()
        # print("a", a, "nulova podminka pro a", vector_to_int_list(a), [0]*dim, vector_to_int_list(a) != [0]*dim)
        # print(vector_to_int_list(a) not in basis_as_list)
        basis_as_list[2] = vector_to_int_list(a)
        basis_as_list[1], basis_as_list[0] = gaussian_reduction(basis_as_list[1], basis_as_list[0])
        basis_as_list[1] = vector_to_int_list(basis_as_list[1])
        basis_as_list[0] = vector_to_int_list(basis_as_list[0])

        basis_as_matrix=matrix(ZZ,basis_as_list)
        G = gram_matrix(basis_as_matrix)
        lc_cube = vector_to_list(find_real_minimum_mistni(G, dim-1, 1))
        print(lc_cube)
        slc = shortest_lc_in_cube(lc_cube, basis_as_matrix)
        # print(slc)
        a = vector(ZZ,slc) * basis_as_matrix
        print(a, "a inside while")

    print("basis as matrix na konci\n", basis_as_matrix)
    basis_as_list.sort(key=lambda x: euclidean_norm(x))

    L = IntegerLattice(basis_as_matrix, lll_reduce=False)
    sv_exact = L.shortest_vector()
    # lc_exact = basis_as_matrix.solve_left(sv_exact)
    sv_exact_norm = sv_exact.norm().n()
    print("\n\nsv exact", sv_exact, sv_exact_norm)
    return euclidean_norm(basis_as_list[0])

def find_real_minimum_mistni(G, current_row, inserted_component) -> vector:
    dimension = len(matrix_to_list(G))
    print(G)
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    print(matrixA, "\n", matrixB)
    result = (matrixA.solve_right(-(inserted_component) * matrixB)).list()
    result.insert(current_row, inserted_component)
    return vector(result).n()


# semaev(randomMatrix(3, 10))
semaev(matrix( [[-10, 7, 4], [-7, 2, -5], [8, -9, -6]]))

[  78  -14  -44]
[ -14  115 -123]
[ -44 -123  181]
[ 78 -14]
[-14 115] 
 [ -44]
[-123]
(-2, -2, -2) a před while

[ 78 -14  20]
[-14 115 -22]
[ 20 -22  12]
[ 78 -14]
[-14 115] 
 [ 20]
[-22]
[-0.22703441987690906, 0.1636653749715067, 1.0]
(-2, -2, -2) a inside while
basis as matrix na konci
 [-7  2 -5]
[-3  5  9]
[-2 -2 -2]


sv exact (-2, -2, -2) 3.46410161513775


3.4641016151377544

In [212]:
 
from math import isclose

def counter(dimension : int, perimeter):
    res = {"True":0, "False":0}
    for i in range(100):
        mojematice = randomMatrix(dimension, perimeter)
        print(mojematice)
        # swapresult = swapping(mojematice)
        swapresult = semaev(mojematice)
        L = IntegerLattice(mojematice, lll_reduce=False)
        sv_exact = L.shortest_vector()
        # lc_exact = basis_as_matrix.solve_left(sv_exact)
        sv_exact_norm = sv_exact.norm().n()
        # LLLresultnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(mojematice.LLL())))
        if  isclose(swapresult, sv_exact_norm, abs_tol=1e-7):
            res["True"]+=1
            continue
        print("falsuje", swapresult)
        res["False"]+=1
    return res

counter(3, 100)

[  3 -77 -39]
[-67  63 -19]
[-18  -2 -64]
[ 4424 -1828  2296]
[-1828  6691 -6607]
[ 2296 -6607  8819]
[ 4424 -1828]
[-1828  6691] 
 [ 2296]
[-6607]
(-46, -12, 6) a před while

[ 4424 -1828   468]
[-1828  6691    84]
[  468    84  2296]
[ 4424 -1828]
[-1828  6691] 
 [468]
[ 84]
[-0.12509577522715676, -0.04673069453224369, 1.0]
(-46, -12, 6) a inside while
basis as matrix na konci
 [-18  -2 -64]
[ 21 -75  25]
[-46 -12   6]


sv exact (-46, -12, 6) 47.9165942028438
[-90  40  69]
[ -5  34 -28]
[-95 -54 -21]
[ 1965  -773  -122]
[ -773 12382  4941]
[ -122  4941 14461]
[ 1965  -773]
[ -773 12382] 
 [-122]
[4941]
(-90, 40, 69) a před while
basis as matrix na konci
 [ -5  34 -28]
[-95 -54 -21]
[-90  40  69]


sv exact (-5, 34, -28) 44.3283205186030
[-57 -43  42]
[ 64  48 -13]
[-56   0  41]
[ 3152   700 -1344]
[  700  4817  4914]
[-1344  4914  6862]
[3152  700]
[ 700 4817] 
 [-1344]
[ 4914]
(7, 5, 29) a před while

[3152  700 1108]
[ 700 4817  797]
[1108  797  915]
[3152  700]
[ 700 4817] 
 [110

{'True': 93, 'False': 7}

In [ ]:
def first_nonzero_norm_vector(vectors):
    for v in vectors:
        if euclidean_norm(v) != 0:
            return v
        
def testik(matice):
    matice = matrix(matice)
    lllnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(matrix(ZZ, matice).LLL())))
    return swapping(matice) == lllnorm

# B_1 = [[-8, -9, 7], [10, 9, -6], [-8, 4, 10]]
# assert testik(B_1)

# B_2 = [[9, -1, -6], [7, -9, -5], [9, 6, -2]]
# assert testik(B_2) # NEJEDE

B_3 = [[3, 10, 4], [-4, 2, 7], [-4, -8, -3]]
assert testik(B_3) # FUNGUJE

B_4 =  [[6, -4, -5], [9, 6, -10], [2, 9, 2]]
assert testik(B_4) # FUNGUJE


B =  [[3, 10, 4], [-4, 2, 7], [-4, -8, -3]]
Basis na začatku: 
  b_ 0   8.306623862918075 [-4, 2, 7]
  b_ 1   9.433981132056603 [-4, -8, -3]
  b_ 2   11.180339887498949 [3, 10, 4]
--------------
[-0.17894736842105263, 1, 1.1263157894736842]
sv (-1, 2, 1) 2.44948974278318
  b_ 0   2.449489742783178 [-1.0, 2.0, 1.0]
  b_ 1   8.306623862918075 [-4, 2, 7]
  b_ 2   9.433981132056603 [-4, -8, -3]
[3.8095238095238093, 1, -0.5238095238095238]
sv (-7.0, 8.0, 10.0) 14.5945195193264
ERROR, LONGER VECTOR
reduced lll [[-1, 2, 1], [-2, -2, 5], [-5, 0, -5]] 2.449489742783178
B =  [[6, -4, -5], [9, 6, -10], [2, 9, 2]]
Basis na začatku: 
  b_ 0   8.774964387392123 [6, -4, -5]
  b_ 1   9.433981132056603 [2, 9, 2]
  b_ 2   14.730919862656235 [9, 6, -10]
--------------
[-0.535131555180926, 1, 0.6946836052742489]
sv (2, 9, 2) 9.43398113205660
ERROR: I already used the vector  (2, 9, 2)
[1, -1.5601193610672284, -1.180270317711076]
sv (-5, -19, 3) 19.8746069143518
ERROR, LONGER VECTOR
[1.1192162188379087, 1,